<a href="https://colab.research.google.com/github/myeongseop2/NIDS_project/blob/main/Probe/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import csv
import collections
import sklearn.metrics as splt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

scaler = MinMaxScaler()
le = LabelEncoder()

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/KDDTrain_probe.csv')
test_df = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/KDDTest_probe.csv')
feature_names = collections.defaultdict(list)

with open('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/Field_Names.txt', 'r') as f:
  for line in f.readlines()[1:]:
    name, nature = line.strip()[:-1].split(':')
    feature_names[nature.replace(" ", "")].append(name)
  
df = pd.concat([train_df, test_df], ignore_index=True)
df.to_csv("/content/drive/MyDrive/project/NIDS/data/NSL_KDD/total.csv", index = False)

In [ ]:
normal_df = df.loc[df['attack_type'] == 'normal']
probe_df = df.loc[df['attack_type'] != 'normal']
normal_df = normal_df.reset_index()
probe_df = probe_df.reset_index()
probe_df

,index,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type,level
0,67343,0,tcp,private,REJ,0,0,0,0,0,...,0.00,0.84,0.00,0.00,0.07,0.00,0.62,1.00,satan,18
1,67344,0,tcp,other,REJ,0,0,0,0,0,...,0.00,0.91,0.00,0.00,0.11,0.00,0.89,1.00,satan,18
2,67345,0,udp,other,SF,1,0,0,0,0,...,0.02,0.06,1.00,0.00,0.00,0.00,0.00,0.00,satan,20
3,67346,0,tcp,private,REJ,0,0,0,0,0,...,0.00,0.80,0.00,0.00,0.03,0.00,0.61,1.00,satan,18
4,67347,0,tcp,private,REJ,0,0,0,0,0,...,0.00,0.47,0.00,0.00,0.02,0.00,0.13,1.00,satan,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14072,91125,0,tcp,pop_3,RSTO,0,36,0,0,0,...,0.19,0.03,0.00,0.00,0.01,0.02,0.73,0.98,mscan,15
14073,91126,0,tcp,pop_3,RSTO,0,36,0,0,0,...,0.25,0.11,0.02,0.03,0.00,0.00,0.86,1.00,mscan,12
14074,91127,5,tcp,imap4,RSTO,0,44,0,0,0,...,0.13,0.04,0.01,0.06,0.00,0.00,0.47,0.97,mscan,14
14075,91128,2,tcp,telnet,SF,24,109,0,0,0,...,0.33,0.03,0.01,0.02,0.00,0.00,0.88,0.69,mscan,8


In [ ]:
attack_category = []
attack_type_int_test = []
attack_type = list(df['attack_type']  == 'normal')
for t in attack_type:
  if t:
    attack_category.append('normal')
    attack_type_int_test.append(0)
  else:
    attack_category.append('probe')
    attack_type_int_test.append(1)

df['attack_category'] = attack_category
df['attack_type'] = attack_type_int_test
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type,level,attack_category
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.03,0.17,0.00,0.00,0.00,0.05,0.00,0,20,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,0.60,0.88,0.00,0.00,0.00,0.00,0.00,0,15,normal
2,0,tcp,http,SF,232,8153,0,0,0,0,...,0.00,0.03,0.04,0.03,0.01,0.00,0.01,0,21,normal
3,0,tcp,http,SF,199,420,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,21,normal
4,0,tcp,http,SF,287,2251,0,0,0,0,...,0.00,0.12,0.03,0.00,0.00,0.00,0.00,0,21,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91125,0,tcp,pop_3,RSTO,0,36,0,0,0,0,...,0.03,0.00,0.00,0.01,0.02,0.73,0.98,1,15,probe
91126,0,tcp,pop_3,RSTO,0,36,0,0,0,0,...,0.11,0.02,0.03,0.00,0.00,0.86,1.00,1,12,probe
91127,5,tcp,imap4,RSTO,0,44,0,0,0,0,...,0.04,0.01,0.06,0.00,0.00,0.47,0.97,1,14,probe
91128,2,tcp,telnet,SF,24,109,0,0,0,0,...,0.03,0.01,0.02,0.00,0.00,0.88,0.69,1,8,probe


In [ ]:
y = df['attack_type']

df = df.drop(['level', 'attack_category'], axis=1)

In [ ]:
under_sampler = RandomUnderSampler()
X, y = under_sampler.fit_resample(df, y)

In [ ]:
y.value_counts()

0    14077
1    14077
Name: attack_type, dtype: int64

In [ ]:
total_col = df.columns
cols = ['duration', 'protocol_type', 'service', 'flag', 'count', 'serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 
       'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_srv_serror_rate', 
       'dst_host_rerror_rate','dst_host_srv_rerror_rate']

for c in total_col:
  if c not in cols:
    X = X.drop(c, axis=1)

In [ ]:
y_df = pd.DataFrame(y, columns = ['attack_type'])
y_df

,attack_type
0,0
1,0
2,0
3,0
4,0
...,...
28149,1
28150,1
28151,1
28152,1


In [ ]:
X.to_csv("/content/drive/MyDrive/project/NIDS/data/NSL_KDD/probe.csv", index = False)
y_df.to_csv("/content/drive/MyDrive/project/NIDS/data/NSL_KDD/probe_attack_type.csv", index = False)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/probe.csv')
test = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/probe_attack_type.csv')

In [ ]:
train

,duration,protocol_type,service,flag,count,serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,http,SF,6,0.0,0.00,0.00,1.00,0.0,0.00,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0,tcp,http,SF,17,0.0,0.00,0.00,1.00,0.0,0.07,255,1.00,0.00,0.01,0.05,0.00,0.00,0.00
2,0,udp,private,SF,307,0.0,0.00,0.00,1.00,0.0,0.00,255,1.00,0.00,0.99,0.00,0.00,0.00,0.00
3,0,tcp,http,SF,1,0.0,0.00,0.00,1.00,0.0,0.00,255,1.00,0.00,0.01,0.01,0.00,0.00,0.00
4,0,tcp,http,SF,37,0.0,0.00,0.00,1.00,0.0,0.00,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28149,0,tcp,pop_3,RSTO,3,0.0,1.00,0.91,0.33,1.0,1.00,49,0.19,0.03,0.00,0.00,0.02,0.73,0.98
28150,0,tcp,pop_3,RSTO,2,0.0,0.50,1.00,0.50,1.0,0.00,60,0.25,0.11,0.02,0.03,0.00,0.86,1.00
28151,5,tcp,imap4,RSTO,2,0.0,1.00,1.00,0.50,1.0,1.00,36,0.13,0.04,0.01,0.06,0.00,0.47,0.97
28152,2,tcp,telnet,SF,3,0.0,0.67,0.00,0.33,1.0,1.00,84,0.33,0.03,0.01,0.02,0.00,0.88,0.69


In [ ]:
test = (test['attack_type'])

In [ ]:
train['protocol_type'] = le.fit_transform(train['protocol_type'])
train['service'] = le.fit_transform(train['service'])
train['flag'] = le.fit_transform(train['flag'])
scaler.fit(train)
train = scaler.transform(train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, test, random_state=42, stratify=test)

In [ ]:
x_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)
x_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)

In [ ]:
x_train.to_csv("/content/drive/MyDrive/project/NIDS/data/NSL_KDD/final/x_train.csv", index=False)
y_train.to_csv("/content/drive/MyDrive/project/NIDS/data/NSL_KDD/final/y_train.csv", index=False)
x_test.to_csv("/content/drive/MyDrive/project/NIDS/data/NSL_KDD/final/x_test.csv", index=False)
y_test.to_csv("/content/drive/MyDrive/project/NIDS/data/NSL_KDD/final/y_test.csv", index=False)

In [ ]:
x_train = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/final/x_train.csv')
y_train = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/final/y_train.csv')
x_test = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/final/x_test.csv')
y_test = pd.read_csv('/content/drive/MyDrive/project/NIDS/data/NSL_KDD/final/y_test.csv')